In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df=pd.read_csv('Titanic.txt')

In [4]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [5]:
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
df.head()

,Survived,Pclass,Sex,Age,Fare
0,0,3,male,22.0,7.2500
1,1,1,female,38.0,71.2833
2,1,3,female,26.0,7.9250
3,1,1,female,35.0,53.1000
4,0,3,male,35.0,8.0500


In [6]:
inputs = df.drop('Survived',axis='columns')
target = df.Survived

In [7]:
dummies = pd.get_dummies(inputs.Sex)
dummies.head(3)

,female,male
0,0,1
1,1,0
2,1,0


In [8]:
inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head(3)

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,0,1
1,1,female,38.0,71.2833,1,0
2,3,female,26.0,7.9250,1,0


## I am dropping male column as well because of dummy variable trap theory. One column is enough to repressent male vs female

In [9]:
inputs.drop(['Sex','male'],axis='columns',inplace=True)
inputs.head(3)

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1


In [10]:
inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

In [11]:
inputs.Age[:10] # we are checking wether we have NaN value or not

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64

In [12]:
# we have to get rid of NaN value by replaing it with mean value
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.head()

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1
3,1,35.0,53.1000,1
4,3,35.0,8.0500,0


In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.3)

In [15]:
len(X_train)

623

In [16]:
len(X_test)

268

In [17]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [19]:

model.fit(X_train,y_train)

GaussianNB()

In [20]:
model.score(X_test,y_test)

0.7611940298507462

In [21]:
X_test[0:10]

,Pclass,Age,Fare,female
358,3,29.699118,7.8792,1
116,3,70.500000,7.7500,0
166,1,29.699118,55.0000,1
612,3,29.699118,15.5000,1
698,1,49.000000,110.8833,0
531,3,29.699118,7.2292,0
599,1,49.000000,56.9292,0
309,1,30.000000,56.9292,1
545,1,64.000000,26.0000,0
851,3,74.000000,7.7750,0


In [22]:
y_test[0:10]

358    1
116    0
166    1
612    1
698    0
531    0
599    1
309    1
545    0
851    0
Name: Survived, dtype: int64

In [23]:
model.predict(X_test[0:10])

array([1, 0, 1, 1, 1, 0, 0, 1, 0, 0], dtype=int64)

In [24]:
model.predict_proba(X_test[:10])

array([[0.35741302, 0.64258698],
       [0.93595359, 0.06404641],
       [0.03285034, 0.96714966],
       [0.36098981, 0.63901019],
       [0.105771  , 0.894229  ],
       [0.96852309, 0.03147691],
       [0.63017297, 0.36982703],
       [0.03119599, 0.96880401],
       [0.67007901, 0.32992099],
       [0.92430847, 0.07569153]])

## Calculate the score using cross validation

In [25]:
from sklearn.model_selection import cross_val_score
cross_val_score(GaussianNB(),X_train, y_train, cv=5)

array([0.808     , 0.856     , 0.72      , 0.75      , 0.76612903])